In [ ]:
import numpy as np

path = '/home/neko/spring2025/scRNA_Compression/data/GSE294399_WBC_020823_matrix.mtx/matrix.mtx'

cells = {}
with open(path, "r") as rf:
    line_number = 1
    for line in rf:
        if line_number >= 4:
            elems = [int(n) for n in line.split()]
            list.append(elems)
            gene = elems[0]
            cell = elems[1]
            count = elems[2]
            
            if cell not in cells:
                cells[cell] = []
        line_number += 1

rf.close()

In [7]:
from sklearn.cluster import KMeans
import numpy as np
X = np.array([[1, 2], [1, 4], [1, 0],
              [10, 2], [10, 4], [10, 0]])
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(X)
print(kmeans.labels_)
kmeans.predict([[0, 0], [12, 3]])
kmeans.cluster_centers_

[1 1 1 0 0 0]


array([[10.,  2.],
       [ 1.,  2.]])

In [9]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from keras.layers import Input, Dense, Layer
from keras.models import Model
from keras import backend as K

# Load data (replace 'cell_gene_data.csv' with your data file)
data = pd.read_csv('cell_gene_data.csv', index_col=0)
X = data.values

# Preprocessing: Log transform and scaling
X = np.log1p(X)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

input_dim = X_scaled.shape[1]
encoding_dim = 32  # Latent space dimension
n_clusters = 3     # Number of clusters (adjust as needed)

# Step 1: Pretrain Autoencoder
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Train autoencoder
autoencoder.fit(X_scaled, X_scaled, epochs=100, batch_size=32, shuffle=True)

# Extract encoder model
encoder_model = Model(inputs=input_layer, outputs=encoder)
encoded_data = encoder_model.predict(X_scaled)

# Step 2: Initialize Cluster Centers using K-means
kmeans = KMeans(n_clusters=n_clusters, n_init=10)
kmeans.fit(encoded_data)
cluster_centers = kmeans.cluster_centers_

# Step 3: Define Clustering Layer
class ClusteringLayer(Layer):
    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights

    def build(self, input_shape):
        self.clusters = self.add_weight(
            shape=(self.n_clusters, input_shape[1]),
            initializer='glorot_uniform',
            name='clusters'
        )
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

# Build Clustering Model
clustering_layer = ClusteringLayer(n_clusters, weights=[cluster_centers], name='clustering')(encoder)
clustering_model = Model(inputs=input_layer, outputs=clustering_layer)
clustering_model.compile(optimizer='adam', loss='kld')

# Step 4: Define Target Distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(axis=0)
    return (weight.T / weight.sum(axis=1)).T

# Train Clustering Model
max_iter = 100  # Number of iterations for refinement
for epoch in range(max_iter):
    if epoch % 10 == 0:
        print(f"Iteration {epoch}")

    # Predict soft assignments
    q = clustering_model.predict(X_scaled, verbose=0)
    p = target_distribution(q)  # Update target distribution

    # Train on current target distribution
    clustering_model.fit(X_scaled, p, epochs=1, batch_size=32, verbose=0)

# Get Final Cluster Assignments
q_final = clustering_model.predict(X_scaled, verbose=0)
cluster_labels = np.argmax(q_final, axis=1)

print("Cluster Assignments:", cluster_labels)

Epoch 1/100


2025-05-05 16:27:02.570102: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-05 16:27:02.774706: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1/1 [==============================] - 0s 294ms/step - loss: 0.1311
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1300
Epoch 3/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1290
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1279
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1269
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1259
Epoch 7/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1249
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1239
Epoch 9/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1230
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1220
Epoch 11/100
1/1 [==============================] - 0s 3ms/step - loss: 0.1211
Epoch 12/100
1/1 [==============================] - 0s 4ms/step - loss: 0.1202
Epoch 13/100
1/1 [==============================] - 0s 4ms/step - loss: